# Cytoscape Reactome FI Example

Overview
--------
This example demonstrates a workflow using ReactomeFIViz CyREST API to perform a comparison pathway and network analysis for TCGA BRCA (breast invasive carcinoma) and OV (ovarian serous cystadenocarcinoma) mutation data.

The steps executed are as follows:

1. Download each cohort mutation annotation file (MAF) from the Broad Firehose WS server.

2. Build the two networks using the ReactomeFIViz CyREST API buildFISubNetwork resource.

3. Cluster the network into network modules using the cluster resource.

4. Perform pair-wise module overlapping analysis.

5. Load the Reactome pathways.

6. Partition the network modules generated from two networks into the following sets:

  * Modules with a significant proportion of genes in common

  * Unshared modules for each of the two cancer types

7. Perform pathway enrichment analysis for each unshared gene module.

8. Export the diagram for an unshared pathway for each cancer type.

Prerequisites
------------
* Python version 2.6 or later.

* Python pandas package version 0.22.0 or later.

* The latest version of Cytoscape with the latest version of the
  ReactomeFIViz app.

Installation
-----------
* Install the required packages in a Python environment, e.g. in the
  [Anaconda](https://docs.anaconda.com/anaconda/) virtual environment:

      conda create -n cytoscape pandas scipy statsmodels
      source activate cytoscape

* Clone the example GitHub repository, e.g.:
  
      mkdir ~/reactome-fi
      cd ~/reactome-fi
      git clone https://github.com/reactome-fi/workflows.git
      cd workflows

* In Cytoscape, select `Help > Check for Updates` to ensure that
  ReactomeFIViz is current.

Usage
-----
* Start Cytoscape.

* In a console, start Jupyter on the notebook directory, e.g.:

      ipython notebook --notebook-dir=~/reactome-fi/workflows

* Run all notebook cells in the web browser Jupyter window.

*Note*: Building the networks clears an existing Cytoscape session,
if it exists.

___

**Utility functions.**

In [ ]:
# Python utilities.
import sys
import os
import re
from functools import reduce
import requests
import itertools
import pandas as pd
from scipy.stats import hypergeom
from statsmodels.sandbox.stats.multicomp import multipletests
from IPython.display import (display, HTML)

In [ ]:
# Constants.

# The number of background genes N is the Reactome FI
# network gene size.
BACKGROUND_GENE_CNT = 12283

# The REST service url.
SERVICE_URL = 'http://localhost:1234'

# The default minimum cluster module size.
DEF_MIN_MODULE_SIZE = 3

In [ ]:
def get_url(*params, **kwargs):
    """
    :param params: the URL path component strings
    :option app: the CyREST application name
    :return: the REST app URL
    """
    path = [SERVICE_URL]
    app = kwargs.get('app')
    if app is not None:
        path.append(app)
    path.append('v1')
    path.extend(params)
    return '/'.join(path)

# Returns the CyREST Reactome FI url.
get_fi_url = lambda *params: get_url(*params, app='reactomefiviz')

In [ ]:
def parse_fi_table_response(resp, parsers, index=None):
    """
    Returns the data frame for the given CyREST Reactome
    FI response. The response `data` property object must
    be a JSON object with properties *tableHeaders* and
    *tableContent*. If the response `data` is empty, then
    this function returns an empty data frame with columns
    given by the *parsers* keys.
    
    The required *parsers* dictionary argument associates
    a parser with each column.
    
    :param resp: the CyREST response
    :param parsers: the column parser dictionary
    :option index: the index column name
    :return: the parsed data frame
    """
    # The response JSON data object.
    data = resp.json()['data']
    # The data columns.
    columns = data.get('tableHeaders') if data else parsers.keys()
    parsers_list = [parsers[col] for col in columns]
    # Parses a content row.
    parse_row = lambda row: tuple(parsers_list[i](value)
                                  for i, value in enumerate(row))
    # The parsed content list.
    content = map(parse_row, data['tableContent']) if data else []
    # Return the data frames.
    return pd.DataFrame.from_records(content, index=index, columns=columns)

In [ ]:
def download_maf(cohort, out=None):
    """
    Downloads the MAF file for the given cancer type cohort.
    The default output file is `<cohort>.maf` in the
    sandbox directory.
    
    *Note*: As of April 2018, the Firebrowse server is unstable.
    Calling this method is not recommended until the server
    stabilizes.
    
    :param cohort: the cancer type cohort name
    :option out: the optional output file path
    :return: the output file path
    """
    # Download the MAF.
    url = 'http://firebrowse.org/api/v1/Analyses/Mutation/MAF'
    maf_file = out if out else os.path.join(sandbox, "%s.maf" % cohort) 
    print("Downloading the %s MAF file to %s..." % (cohort, maf_file))
    params = dict(format='tsv', cohort=cohort, page_size=200)
    eof = False
    page = 1
    with open(maf_file, 'w') as f:
        while not eof:
            params['page'] = page
            resp = requests.get(url, params=params)
            if not resp.ok:
                print("Error encountered downloading the %s MAF file. Please retry." %
                      cohort)
                resp.raise_for_status()
            text = resp.text
            if text:
                f.write(text)
                page = params['page'] = page + 1
            else:
                eof = True
            print('+', end='')
    print('')
    print("MAF file downloaded.")
    return maf_file

In [ ]:
def build_network(maf_file, cutoff=.01):
    """
    Builds the network from the given MAF file.
    Only gene modules whose sample size exceeds a
    threshold are included. The threshold is the
    greater of 2 and sample count * cutoff, where
    sample count is the total number of MAF samples.
    
    :param maf_file: the downloaded MAF file
    :option cutoff: the minimum proportion of samples
    """
    # Clear the current Cytoscape session, if any.
    requests.delete(get_url('session')).ok
    # Read the MAF into a data frame.
    maf_df = pd.read_csv(maf_file, sep='\t',
                         usecols=['Tumor_Sample_Barcode'])
    # The number of samples.
    sample_cnt = len({tsb for tsb in maf_df.Tumor_Sample_Barcode})
    print("Sample Count: %d" % sample_cnt)
    sample_cutoff = max(2, int(0.01*sample_cnt))
    print("Building the FI network with sample cut-off %d..." %
          sample_cutoff)
    # Build the FI network with 1% sample cut-off.
    body = dict(fiVersion='2016', format='MAF', file=maf_file,
                enteredGenes='', chooseHomoGenes=False,
                userLinkers=False, showUnLinked=False,
                fetchFIAnnotations=True,
                sampleCutoffValue=sample_cutoff)
    requests.post(get_fi_url('buildFISubNetwork'), json=body)
    print("The FI network is loaded to Cytoscape.")
    return maf_df

In [ ]:
# The cluster content value parsers.
cluster_parsers = {
    'Module': int, 'Nodes in Module': int, 'Node Percentage': float,
    'Samples in Module': int, 'Sample Percentage': float,
    'Node List': lambda s: s.split(',')
}

def cluster():
    """
    Cluster the network currently displayed in Cytoscape into
    gene modules.
    
    :return: a data frame with index *Module* and columns
      *module_size* and *genes*
    """
    # Cluster the currently displayed network.
    print("Clustering the FI network...")
    resp = requests.get(get_fi_url('cluster'))
    # Parse the response JSON into a data frame.
    parsed = parse_fi_table_response(resp, parsers=cluster_parsers,
                                     index='Module')
    # Rename the columns.
    rename_opts = {'Node List': 'Genes', 'Nodes in Module': 'Module Size'}
    renamed = parsed.rename(columns=rename_opts)
    # Retain only the genes and size columns.
    sliced = renamed.loc[:, ['Module Size', 'Genes']]
    print("The cluster table is available in Cytoscape.")
    return sliced

In [ ]:
def prepare(cohort):
    """
    Prepares the given cancer type cohort for analysis.
    
    :param cohort: the cancer type cohort name
    :return: the clustered data frame
    """
    # Clear the current Cytoscape session, if any.
    try:
        requests.delete(get_url('session')).ok
    except requests.exceptions.ConnectionError:
        print("Error: Connection refused: is Cytoscape started?")
        raise StandardError()
    maf_file = os.path.join(sandbox, "%s.maf" % cohort)
    # Download the MAF, if necessary.
    if (not os.path.exists(maf_file)):
        # Download disabled due to Firebrowse server instability.
        #download_maf(cohort, out=maf_file)
        raise IOError("The %s MAF file was not found: %s" %
                      (cohort, maf_file)) 
    # Build the network.
    build_network(maf_file)
    # Cluster into gene modules.
    return cluster()

In [ ]:
def overlap_pvalue(N, overlap, gs1, gs2):
    """
    :param N: the number of background genes
    :param overlap: the genes in common
    :param gs1: the first gene set
    :param gs2: the second gene set
    :return: the probability of gene set overlap from
      a background population of *N* genes
    """
    # The hypergeometric distribution parameters.
    k = len(overlap) - 1
    n1 = len(gs1)
    n2 = len(gs2)
    # Return the overlap probability.
    return hypergeom.sf(k, N, n1, n2)

In [ ]:
def filter_on_module_size(inputs, **kwargs):
    """
    Returns subsets of the given data frames with module
    count no greater than a threshold value. The threshold
    is given by the *max_module_count* option, with default
    the minimum module count of the input data frames.
    The module selection criterion is a gene module size
    which ensures that no more than *max_module_count*
    modules are selected from each data frame.
    
    :param inputs: the input {name: data frame} dictionary
    :param kwargs: the following options:
    :option min_module_size: the minimum module size (default 3)
    :option max_module_count: the maximum number of modules
    :return: the corresponding filtered data frames
    """
    # The size cutoff.
    max_module_cnt_opt = kwargs.get('max_module_count')
    if max_module_cnt_opt:
        n = max_module_cnt_opt
    else:
        n = min(len(d) for df in modules)
    min_module_size_opt = kwargs.get('min_module_size')
    if min_module_size_opt:
        min_module_size = min_module_size_opt
    else:
        min_module_size = DEF_MIN_MODULE_SIZE
    # The n largest data frames.
    largest = [df['Module Size'].nlargest(n) for df in inputs.values()]
    sizes = [ds.iloc[n - 1] for ds in largest]
    cutoff = max(min_module_size, *sizes)
    return {name: df.loc[df['Module Size'].ge(cutoff), :]
            for name, df in inputs.items()}

def slice_on_node_list(name, df):
    """
    Slices the given data frame vertically on the *genes* column.
    Renames the *Module* index to the given name and the *genes*
    column to '<name> Genes', e.g. 'BRCA Genes' for *name* 'BRCA'.
    Adds a column '<name> Size' with the module size.
    Enriches the genes and adds a column'
    
    :param name: the cancer type cohort name
    :param df: the gene modules data frame
    :return: the data frame with renamed index and column
    """
    rename_dict = {'Genes': "%s Genes" % name}
    sliced_df = df.loc[:, ['Genes']].rename(columns=rename_dict)
    sliced_df.index.names = [name]
    size_col = "%s Size" % name
    size = df.Genes.apply(len)
    return sliced_df.assign(**{size_col: size})

def cartesian(df1, df2):
    """
    Takes the cartesian product of the input data frames.
    The resulting data frame has a mult-index with levels
    given by the respective input data frame indexes.
    
    :param df1: the first data frame
    :param df2: the second data frame
    :return: the cartesian product of the input data frames
    """
    rows = itertools.product(df1.iterrows(), df2.iterrows())
    values = (left.append(right) for (_, left), (_, right) in rows)
    indexes = [df1.index, df2.index]
    index_names = [index.name for index in indexes]
    index_values = [index.values for index in indexes]
    multi_index = pd.MultiIndex.from_product(index_values, names=index_names)
    return pd.DataFrame(values, index=multi_index)

In [ ]:
def append_overlap(n, cross_df):
    """
    Calculates the pair-wise overlap p-value for the given
    gene module cross-product.
    The return value is the cross-product augmented with
    two columns:
    * _p-value_ - the hypergeometric test p value
    * _FDR_ - the FDR corrected for multiple tests
    
    :param n: the background gene count
    :param cross_df: the cross-product data frame
    :return: the augmented data frame
    """
    # Determines the shared genes.
    intersect = lambda gs1, gs2: set(gs1).intersection(set(gs2))
    # Select only the gene columns.
    gene_cols = [col for col in cross_df.columns if col.endswith("Genes")]

    # The gene set intersections for each row.
    genesets = lambda row: [row[col] for col in gene_cols]
    shared = [intersect(*genesets(row))
              for _, row in cross_df.iterrows()]
    shared_sizes = [len(gs) for gs in shared]
    # Augment the cross-product with the shared genes and gene count.
    kwargs = {'Shared Genes': shared, 'Shared Size': shared_sizes}
    shared_df = cross_df.assign(**kwargs)

    # Augment the shared data frame with the pvalues.
    pvalue = lambda row: overlap_pvalue(BACKGROUND_GENE_CNT,
                                        row['Shared Genes'],
                                        *genesets(row))
    pvals = shared_df.agg(pvalue, axis=1)
    overlap_df = shared_df.assign(**{'p-value': pvals})

    # Correct the p-values for multiple comparison hypothesis
    # testing by applying the Benjamini–Hochberg FDR procedure.
    _, corrected, _, _ = multipletests(pvals.values, method='fdr_bh')
    overlap_df['FDR'] = corrected
    return overlap_df

def partition_shared(overlap_df, inputs, cutoff=.01):
    """
    Partitions the gene modules into three data series:
    * A shared data series for gene modules with significant
      overlap
    * Unshared data series for each cancer type
    The shared data series has a multi-index of the cancer types.
    The unshared data series has index `Module`.
    Each data series values are gene lists.
    The gene modules are filtered by p-value (see *cutoff* option below).
    
    :param overlap_df: the input data frame is the BRCA x OV
      module cartesian cross-product augmented with a pair-wise
      overlap *FDR*
    :param inputs: the {name: <data frame>} inputs
    :option cutoff: the corrected p-value cutoff value which
      determines a shared module (default .01)
    :return the {'shared': <series>, 'unshared': {name: <series>}}
      data series dictionary for each *name* in inputs
    """
    # The shared gene modules have FDR <= cutoff
    criterion = overlap_df['FDR'].le(cutoff)
    shared_df = overlap_df.loc[criterion]

    all_modules = [set(values)
                   for values in zip(*overlap_df.index.values)]
    shared_modules = [set(values)
                      for values in zip(*shared_df.index.values)]
    unshared_modules = [all_modules[i].difference(shared_modules[i])
                        for i in range(len(all_modules))]
    unshared_idx_dict = {name: unshared_modules[i]
                         for i, name in enumerate(overlap_df.index.names)}
    unshared = {name: df.loc[unshared_idx_dict[name], :]
                for name, df in inputs.items()}
    for name, df in unshared.items():
        print("Unshared %s module count: %d" % (name, df.index.size))

    return dict(shared=shared_df, unshared=unshared)    

In [ ]:
def discover_overlap(n, inputs, **kwargs):
    """
    Performs pair-wise overlap analysis on the given cluster
    data frames.
    
    :param n: the background gene count
    :param inputs: the {name: data frame} inputs dictionary
    :param kwargs: the filter_on_module_size options 
    :return the {'shared': <series>, 'unshared': {name: <series>}}
      data series dictionary for each *name* in inputs
    """
    # Select the largest modules.
    filtered = filter_on_module_size(inputs, **kwargs)
    # Prep pair-wise analysis by taking the cartesian product
    # of the two filtered data frames renamed with unique columns.
    sliced = {name: slice_on_node_list(name, df)
              for name, df in filtered.items()}
    cross_df = cartesian(*sliced.values())
    overlap_df = append_overlap(n, cross_df)
    return overlap_df

def print_overlap(overlap_df, cutoff=None, format='html'):
    """
    Represents the given overlap data frame as a data frame
    suitable for printing.
    
    :param overlap_df: the raw overlap data frame
    :option cutoff: the FDR cut-off value
    :option format: 'html' or 'text' (default is html)
    :return: a data frame suitable for printing
    """
    # Filter the overlap FDR. Set this to a lower value to
    # Restrict the table size.
    if cutoff:
        filtered_df = overlap_df[overlap_df.FDR.le(cutoff)]
    else:
        filtered_df = overlap_df
    rename_opts = {name: "%s Module" % name
                   for name in filtered_df.index.names}
    flat_df = filtered_df.reset_index().rename(rename_opts, axis=1)

    columns = [col for col in flat_df.columns if not col.endswith("Genes")]
    cohort_col_grps = [[col for col in columns if col.startswith(name)]
                       for name in filtered_df.index.names]
    cohort_cols = reduce(lambda x,y: x + y, cohort_col_grps)
    non_cohort_cols = [col for col in columns if col not in cohort_cols]
    ordered_cols = cohort_cols + non_cohort_cols
    printable_df = flat_df.loc[:, ordered_cols]
    if format == 'text':
        print(printable_df.to_string(index=False))
    elif format == 'html':
        start = '<h4>Module Overlap'
        if cutoff:
            style = 'font-size:normal;font-weight:normal;'
            middle = "<span style=%s> (FDR <= %s)</span>" % (style, cutoff)
        else:
            middle = ''
        end = '</h4>'
        heading = start + middle + end
        display(HTML(heading))
        display(HTML(printable_df.to_html(index=False)))
    else:
        raise ValueError("Unrecognized overlap print format: %s" % format)

In [ ]:
enrichment_parsers = {
    'ReactomePathway': lambda p: p,
    'RatioOfProteinInPathway': float,
    'NumberOfProteinInPathway': int,
    'ProteinFromGeneSet': int,
    'P-value': float,
    'FDR': float,
    'HitGenes': lambda s: s.split(',')
}

def enrich_genes(genes, cutoff=.001):
    """
    Perform pathway enrichment analysis on the given genes.
    The resulting data frame has index *Pathway* and
    columns *p-value* and *FDR*
    
    :param genes: the gene list or set to enrich
    :option cutoff: the FDR cut-off (default .001)
    :return: the result data frame, or an empty data frame
      if the genes could not be enriched
    """
    data = ','.join(genes)
    # Perform the Reactome enrichment analysis.
    resp = requests.post(get_fi_url('ReactomePathwayEnrichment'),
                         data=data)
    resp.raise_for_status()
    parsed_df = parse_fi_table_response(resp, enrichment_parsers,
                                     index='ReactomePathway')
    # Rename the index and P-value column for consistency.
    parsed_df.index.rename('Pathway', inplace=True)
    renamed_df = parsed_df.rename({'P-value': 'p-value'}, axis=1)
    # We only want the p-value and FDR.
    sliced_df = renamed_df.loc[:, ['p-value', 'FDR']]
    # Sort by FDR.
    sorted_df = sliced_df.sort_values(by='FDR')
    # Apply the cutoff, if it exists.
    return sorted_df[sorted_df.FDR.le(cutoff)] if cutoff else sorted_df

def enrich_modules(name, genes_ds, **kwargs):
    """
    Perform pathway enrichment on the given gene modules.
    
    :param name: the cohort name
    :param genes_ds: the data series with module number index
      and gene list value
    :param kwargs: the `enrich_genes` options
    :return: a data frame with Module index and columns
      *Genes* and *Pathways*, where each *Pathways* value
      is the enrichment result data frame
    """
    print("Enriching %d %s modules..." % (genes_ds.index.size, name))
    # Gets the enriched pathways.
    enrich = lambda genes: enrich_genes(genes, **kwargs)
    pathways = genes_ds.apply(enrich)
    print("%s enrichment completed." % name)
    return genes_ds.to_frame().assign(Pathways=pathways)

def _accum_pathways(accum, index, row):
    pathways_df = row.Pathways
    pathways = pathways_df.index.get_values()
    dup_module = [index] * pathways_df.index.size
    multi_tuples = list(zip(pathways, dup_module))
    multi_names = ('Pathway', 'Module')
    multi_index = pd.MultiIndex.from_tuples(multi_tuples, names=multi_names)
    multi_values = pathways_df.values #list(zip(*pathways_df.values))
    multi_cols = pathways_df.columns.get_values()
    multi_df = pd.DataFrame(data=multi_values, columns=multi_cols,
                            index=multi_index)
    return multi_df if accum.empty else accum.append(multi_df)

def transpose_pathways(enrichment_df):
    accumulate = lambda accum, iter_tuple: _accum_pathways(accum, *iter_tuple)
    transposed_df = reduce(accumulate, enrichment_df.iterrows(),
                           pd.DataFrame({'Dummy' : []}))
    return transposed_df

def distinct_pathways(inputs):
    """
    Returns the distinct pathways from the given input data sets.
    
    :param inputs: the input {name, partition data frame} dictionary
    :return: the distinct {name: pathways data series} dictionary
    """
    extract_pathways = lambda table: set(table.index.get_values())
    pathway_ds_dict = {name: df.Pathways.apply(extract_pathways)
                       for name, df in inputs.items()}
    pathways_dict = {name: reduce(lambda s1, s2: s1.union(s2), p)
                     for name, p in pathway_ds_dict.items()}
    for name, pathways in pathways_dict.items():
        print("%s has %d pathways." % (name, len(pathways)))

    intersect = lambda s1, s2: s1.intersection(s2)
    common = reduce(intersect, pathways_dict.values())
    print("There are %d pathways in common between %s and %s." %
          (len(common), *inputs.keys()))

    distinct_dict = {name: sorted(pathways.difference(common))
                for name, pathways in pathways_dict.items()}
    for name, pathways in distinct_dict.items():
        print("%s has %d distinct unshared pathways." %
              (name, len(pathways)))
    return distinct_dict

In [ ]:
def get_hierarchy_node(pathway, hierarchy):
    """
    Returns the Reactome hierarchy node for the given pathway.
    
    :param pathway: the pathway to check
    :param hierarchy: the Reactome pathway hierarchy to check
    :return: the hierarchy node
    """
    if hierarchy['name'] == pathway:
        return hierarchy
    children = hierarchy.get('children')
    if children:
        for subtree in children:
            target = get_hierarchy_node(pathway, subtree)
            if target:
                return target

def export_diagram(db_id, pathway, genes, out_dir=None):
    """
    Exports a diagram PDF for the given pathway. The PDF
    is placed in the target output directory. The file name
    capitalizes the pathway name and removes spaces and
    punctuation, e.g. `DEKBindsTFAP2Homodimers`.
    
    :param db_id: the Reactome pathway db id
    :param pathway: the pathway name to export
    :param genes: the hit genes for the pathway
    :option out_dir: the target directory (default current directory)
    :return: the exported PDF file name
    """
    # Re-enrich the genes in order to get the proper diagram
    # highlighting.
    enrich_genes(genes)
    if not out_dir:
        out_dir = os.getcwd()
    separator = re.compile('[^\w]+')
    capitalized = [word[0].upper() + word[1:]
                   for word in separator.split(pathway) if word]
    base_name = ''.join(capitalized)
    file_name = os.path.join(out_dir, "%s.pdf" % base_name)
    body = dict(dbId=db_id, pathwayName=pathway, fileName=file_name)
    requests.post(get_fi_url('exportPathwayDiagram'), json=body)
    print("Exported pathway '%s' to %s." % (pathway, file_name))
    return file_name

def export_first_pathway_diagram(pathways, df, hierarchy):
    """
    Exports the diagram for the first pathway which has one.
    
    :param pathways: the pathways to choose from
    :param df: the enrichment data frame
    :param hierarchy: the Reactome pathway hierarchy
    :return: the exported diagram file name
    """
    for pathway in pathways:
        node = get_hierarchy_node(pathway, hierarchy)
        if node and node['hasDiagram']:
            db_id = node['dbId']
            genes = next(
                row.Genes for _, row in df.iterrows()
                if pathway in row.Pathways.index
            )
            return export_diagram(db_id, pathway, genes, sandbox)

___

**Prepare the network.**

* Download the cohort mutation annotation file (MAF) from the Broad Firehose WS server.

* Build the network using the ReactomeFIViz CyREST API buildFISubNetwork resource.

* Cluster the network into network modules using the cluster resource.

(cf. User Guide
[Mutation Analysis](http://wiki.reactome.org/index.php?title=ReactomeFIViz&redirect=no#Gene_Set.2FMutation_Analysis)
step)

In [ ]:
# The sandbox directory contains the MAF files and
# generated diagrams. The default is the working
# directory. Change this to your workflows git
# repository location if you don't start the
# notebook in that directory.
sandbox = os.getcwd()

In [ ]:
# Prepare the BRCA cohort for analysis.
# This takes a while.
brca_df = prepare('BRCA')
print("BRCA module count: %d" % brca_df.index.size)

*Expected Result*: Cytoscape displays the BRCA FI network
and clustered gene modules table.

In [ ]:
# Prepare the OV cohort for analysis.
# This takes a while.
ov_df = prepare('OV')
print("OV module count: %d" % ov_df.index.size)

*Expected Result*: Cytoscape displays the OV FI network
and clustered gene modules table.

___

**Perform pair-wise module overlapping analysis.**

Determine the significance of pair-wise module overlap.

In [ ]:
# The analysis input.
inputs = {'BRCA': brca_df, 'OV': ov_df}

# Discover cluster modules with significant overlap.
# Reset the options below to change the number of modules.
# Note that the modules are already filtered by sample
# size during clustering.
module_opts = dict(max_module_count=20, min_module_size=3)
overlap_df = discover_overlap(BACKGROUND_GENE_CNT, inputs,
                              **module_opts)

# Make a data frame suitable for printing.
# Reset the FDR *cutoff* to limit the number of modules
# printed, or set to None to print all modules.
print_overlap(overlap_df, cutoff=.01)

___

**Partition the modules.**

Partition the network modules into the following sets:

* Modules with a significant proportion of genes in common

* Unshared modules for each of the two cancer types

In [ ]:
# Partition into shared and unshared.
partition = partition_shared(overlap_df, inputs)

___

**Load the Reactome pathways.**

(cf. the User Guide
[Explore Pathways](http://wiki.reactome.org/index.php?title=ReactomeFIViz&redirect=no#Explore_Reactome_Pathways)
step)

In [ ]:
resp = requests.get(get_fi_url('pathwayTree'))
reactome_tree = resp.json()['data']

*Expected Result*: Reactome heirarchy displayed in the Cytoscape Control Panel.

___

**Perform pathway enrichment on the unshared gene modules.**

(cf. User Guide
[Enrichment Analysis](http://wiki.reactome.org/index.php?title=ReactomeFIViz&redirect=no#Pathway_Enrichment_Analysis)
step)

In [ ]:
# Perform pathway enrichment on the gene modules.
rename_opts = {'Shared Genes': 'Genes'}
columns = ['Genes', 'p-value', 'FDR']
shared_df = partition['shared'].rename(columns=rename_opts).loc[:, columns]
groups = dict(Shared=shared_df)
groups.update(partition['unshared'])
enriched = {name: enrich_modules(name, df.Genes)
            for name, df in groups.items()}
enriched_unshared = {name: enriched[name] for name in inputs.keys()}
distinct = distinct_pathways(enriched_unshared)

In [ ]:
# Show the unshared pathways. 
display(HTML('<h4>Unshared Module Pathways:</h4>'))
for name, pathways in distinct.items():
    display(HTML("<h5>%s:</h5>" % name))
    transposed_df = transpose_pathways(enriched[name])
    grouped_df = transposed_df.groupby(level='Pathway', sort=False).min()
    grouped_df.sort_values(by='FDR', inplace=True)
    criterion = grouped_df.index.isin(pathways)
    distinct_df = grouped_df[criterion]
    display(HTML(distinct_df.reset_index().to_html(index=False)))

*Expected Result*: Enrichment results are successively displayed in the Cytoscape Table Panel.

In [ ]:
# The cancer type.
cancer = 'BRCA'

# The enriched BRCA module numbers.
module_numbers = enriched[cancer].index.get_values()
display(HTML('<h4>Enriched %s module numbers:</h4>'% cancer))
print(module_numbers)

# Show the enriched pathways for a BRCA module.
# Change the module number to see a different module.
module_number = module_numbers[0]
df = enriched[cancer].loc[module_number]
display(HTML("<h4>Module %d Genes:</h4>" % module_number))
print(','.join(df.Genes))
print()
display(HTML("<h4>Module %d Pathways:</h4>" % module_number))
display(HTML(df.Pathways.loc[:, ['p-value', 'FDR']].reset_index().to_html(index=False)))

___

**Export an unshared pathway diagram for each cancer type.**

In [ ]:
# Export one diagram per cancer type.
for name, pathways in distinct.items():
    print("Exporting a %s pathway..." % name)
    df = enriched[name]
    export_first_pathway_diagram(pathways, df, reactome_tree)